In [94]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

def connect():
    scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
    
    creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
    client = gspread.authorize(creds)    
    sheet = client.open("SwingSummary").worksheet('Summary')  # Open the spreadhseet
    return sheet


def addInvestmentCall(row):
    sheet = connect()
    df = pd.DataFrame(sheet.get_all_records())
    #df['StockName'] = 'test'
    df.loc[len(df)] = row
    sheet.update([df.columns.values.tolist()] + df.values.tolist())
    
    
def updateInvestmentCall(data):
    sheet = connect()
    df = pd.DataFrame(sheet.get_all_records())
    
    
data = '''Investment update:
Security: ISEC
Event: T1 achievedACA
CMP: 500
x
y'''
datad = {}
for i in data.split('\n')[1:4]:
    datad[i.split(':')[0].lower()] = i.split(':')[1].strip()
sheet = connect()        
df = pd.DataFrame(sheet.get_all_records())
df
mask = (df['StockName']==datad['security']) & (df['Status']=='Open')

In [55]:
df = df.loc[mask]

In [57]:
df.loc[(df['StockName']==datad['security']) & (df['Status']=='Open'),'Target1'] = 809

In [58]:
import datetime
df.loc[(df['StockName']==datad['security']) & (df['Status']=='Open'),'Target1Date'] = str(datetime.datetime.now().strftime('%Y-%m-%d'))

In [59]:
df

,StockName,CallDate,BuyPrice,StopLoss,Targets,Target1,Target1Date,Target2,Target2Date,Target3,Target3Date,Time,MonthlyROI,TotalROI,Status,Remarks
3,ISEC,2021-10-19,500,400,,809,2021-10-20,700,,800,,,,,Open,


In [35]:
df[(df['StockName']==datad['security']) & (df['Status']=='Open')]['Time']=

,StockName,CallDate,BuyPrice,StopLoss,Targets,Target1,Target1Date,Target2,Target2Date,Target3,Target3Date,Time,MonthlyROI,TotalROI,Status,Remarks
3,ISEC,2021-10-19,500,400,,809.0,2021-10-20,700,,800,,,,,Open,


In [37]:
df=df[(df['StockName']==datad['security']) & (df['Status']=='Open')]

In [61]:
df.loc[mask, 'Time'] = str(datetime.datetime.strptime(df['Target1Date'].tolist()[0],'%Y-%m-%d')-datetime.datetime.strptime(df['CallDate'].tolist()[0],'%Y-%m-%d')).split(' ')[0]

In [62]:
df


,StockName,CallDate,BuyPrice,StopLoss,Targets,Target1,Target1Date,Target2,Target2Date,Target3,Target3Date,Time,MonthlyROI,TotalROI,Status,Remarks
3,ISEC,2021-10-19,500,400,,809,2021-10-20,700,,800,,1,,,Open,


In [64]:
df.loc[mask,'TotalROI'] = ((float(df['Target1'])-float(df['BuyPrice']))/float(df['BuyPrice']))*100.0

In [70]:
import math
df.loc[mask,'MonthlyROI'] = float(df['TotalROI'])/(math.ceil(float(df['Time'])/30.0))

In [71]:
df

,StockName,CallDate,BuyPrice,StopLoss,Targets,Target1,Target1Date,Target2,Target2Date,Target3,Target3Date,Time,MonthlyROI,TotalROI,Status,Remarks
3,ISEC,2021-10-19,500,400,,809,2021-10-20,700,,800,,1,61.8,61.8,Open,


In [74]:
df

,StockName,CallDate,BuyPrice,StopLoss,Targets,Target1,Target1Date,Target2,Target2Date,Target3,Target3Date,Time,MonthlyROI,TotalROI,Status,Remarks
3,ISEC,2021-10-19,500,400,,600,,700,,800,,,,,Open,


In [76]:
df = pd.DataFrame(sheet.get_all_records())
mask = (df['StockName']==datad['security']) & (df['Status']=='Open')
df = df.loc[mask]


In [77]:
df

,StockName,CallDate,BuyPrice,StopLoss,Targets,Target1,Target1Date,Target2,Target2Date,Target3,Target3Date,Time,MonthlyROI,TotalROI,Status,Remarks
3,ISEC,2021-10-19,500,400,,600,,700,,800,,,,,Open,


In [84]:
data = '''Investment update:
Security: ISEC
Event: T1 achieved
CMP: 600
x
y'''
datad = {}
for i in data.split('\n')[1:4]:
    datad[i.split(':')[0].lower()] = i.split(':')[1].strip()
sheet = connect()        
df = pd.DataFrame(sheet.get_all_records())
mask = (df['StockName']==datad['security']) & (df['Status']=='Open')
df = df.loc[mask]
dataList = [datad['cmp'], str(datetime.datetime.now().strftime('%Y-%m-%d')),
            str(datetime.datetime.now()-datetime.datetime.strptime(df['CallDate'].tolist()[0],'%Y-%m-%d')).split(' ')[0],
            ((float(datad['cmp'])-float(df['BuyPrice']))/float(df['BuyPrice']))*100.0,
            float(((float(datad['cmp'])-float(df['BuyPrice']))/float(df['BuyPrice']))*100.0)/(math.ceil(float(df['Time'])/30.0)),
            'Target 1 Achieved'
            ]

df.loc[mask,['Target1', 'Target1Date', 'Time', 'TotalROI', 'MonthlyROI', 'Remarks']]=dataList

ValueError: could not convert string to float: ''

In [128]:
data = '''Investment update:
Security: ISEC2
Event: T1 achieved
CMP: 300
x
y'''
datad = {}
for i in data.split('\n')[1:4]:
    datad[i.split(':')[0].lower()] = i.split(':')[1].strip()
sheet = connect()        
df = pd.DataFrame(sheet.get_all_records())
#print(df.loc[mask])
mask = (df['StockName']==datad['security']) & (df['Status']=='Open')
dataList = [datad['cmp'], str(datetime.datetime.now().strftime('%Y-%m-%d')),
                    str(datetime.datetime.now()-datetime.datetime.strptime(df.loc[mask,'CallDate'].tolist()[0],'%Y-%m-%d')).split(' ')[0] if 'day' in str(datetime.datetime.now()-datetime.datetime.strptime(df.loc[mask,'CallDate'].tolist()[0],'%Y-%m-%d')) else 1.0,
                    ((float(datad['cmp'])-float(df.loc[mask,'BuyPrice']))/float(df.loc[mask,'BuyPrice']))*100.0,
                    float(((float(datad['cmp'])-float(df.loc[mask,'BuyPrice']))/float(df.loc[mask,'BuyPrice']))*100.0)/(math.ceil(float(str(datetime.datetime.now()-datetime.datetime.strptime(df.loc[mask,'CallDate'].tolist()[0],'%Y-%m-%d')).split(' ')[0] if 'day' in str(datetime.datetime.now()-datetime.datetime.strptime(df.loc[mask,'CallDate'].tolist()[0],'%Y-%m-%d')) else 1.0)/30.0))
                    ,'g']
df.loc[mask,['Target1', 'Target1Date', 'Time', 'TotalROI', 'MonthlyROI', 'Remarks']]=dataList

In [129]:
df

,StockName,CallDate,BuyPrice,StopLoss,SLDate,Targets,Target1,Target1Date,Target2,Target2Date,Target3,Target3Date,Time,MonthlyROI,TotalROI,Status,Remarks
0,ISEC,2021-10-19,500,400,,,500,2021-10-20,600,2021-10-20,700,2021-10-20,1,40,40,Closed,Target3 Achieved
1,ISEC2,2021-10-20,500,400,,600/700/800,300,2021-10-20,,,,,1,-40,-40,Open,g
2,ISEC3,2021-10-20,500,400,,600/700/800,,,,,,,,,,Open,


In [112]:
df

,StockName,CallDate,BuyPrice,StopLoss,SLDate,Targets,Target1,Target1Date,Target2,Target2Date,Target3,Target3Date,Time,MonthlyROI,TotalROI,Status,Remarks
0,ISEC,2021-10-19,500,400,,,600,2021-10-20,700,,800,,1,20,20,Open,Target 1 Achieved


In [120]:
a = datetime.datetime.now()
import time
time.sleep(10)
print(datetime.datetime.now()-a)


0:00:10.000701


In [130]:
import requests

In [132]:
a=requests.get('https://archives.nseindia.com/content/historical/EQUITIES/2021/OCT/cm20OCT2021bhav.csv.zip')

In [135]:
import pandas as pd

In [138]:
df = pd.read_csv('https://archives.nseindia.com/content/historical/EQUITIES/2021/OCT/cm20OCT2021bhav.csv.zip')
https://archives.nseindia.com/products/content/sec_bhavdata_full_20102021.csv
    https://archives.nseindia.com/content/indices/ind_close_all_20102021.csv

In [139]:
df

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,Unnamed: 13
0,20MICRONS,EQ,61.40,63.30,59.95,62.80,63.15,61.35,157664,9.733488e+06,20-OCT-2021,2584,INE144J01027,NaN
1,21STCENMGM,EQ,49.10,51.10,49.10,49.35,49.90,50.10,83830,4.154268e+06,20-OCT-2021,403,INE253B01015,NaN
2,3MINDIA,EQ,25720.00,26549.75,25283.70,26016.70,26000.00,25981.20,6201,1.606716e+08,20-OCT-2021,3927,INE470A01017,NaN
3,3PLAND,EQ,13.45,14.00,13.20,13.30,13.30,13.40,7686,1.031158e+05,20-OCT-2021,84,INE105C01023,NaN
4,585GS2030,GS,98.63,98.63,98.63,98.63,98.63,98.63,200,1.972600e+04,20-OCT-2021,1,IN0020200294,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2115,ZOMATO,EQ,138.50,140.40,133.85,137.90,138.20,138.55,12607297,1.716424e+09,20-OCT-2021,107977,INE758T01015,NaN
2116,ZOTA,EQ,385.10,391.25,377.50,381.30,382.00,391.75,60683,2.319778e+07,20-OCT-2021,2584,INE358U01012,NaN
2117,ZUARI,EQ,121.00,122.00,117.00,118.45,118.20,121.65,31257,3.734510e+06,20-OCT-2021,666,INE840M01016,NaN
2118,ZUARIGLOB,EQ,139.60,139.60,129.85,132.75,132.05,134.75,117194,1.550330e+07,20-OCT-2021,434,INE217A01012,NaN


In [ ]:
https://www1.nseindia.com/content/indices/ind_niftyhealthcarelist.csv
https://www1.nseindia.com/content/indices/ind_niftyautolist.csv
https://www1.nseindia.com/content/indices/ind_niftybanklist.csv
https://www1.nseindia.com/content/indices/ind_niftyconsumerdurableslist.csv
https://www1.nseindia.com/content/indices/ind_niftyfinancelist.csv
https://www1.nseindia.com/content/indices/ind_niftyfinancialservices25_50list.csv
https://www1.nseindia.com/content/indices/ind_niftyfmcglist.csv
https://www1.nseindia.com/content/indices/ind_niftyitlist.csv
https://www1.nseindia.com/content/indices/ind_niftymedialist.csv
https://www1.nseindia.com/content/indices/ind_niftymetallist.csv
https://www1.nseindia.com/content/indices/ind_niftyoilgaslist.csv
https://www1.nseindia.com/content/indices/ind_niftypharmalist.csv
https://www1.nseindia.com/content/indices/ind_nifty_privatebanklist.csv
https://www1.nseindia.com/content/indices/ind_niftypsubanklist.csv
https://www1.nseindia.com/content/indices/ind_niftyrealtylist.csv
NIFTY_SME_EMERGE:https://www1.nseindia.com/content/indices/ind_niftysmelist.csv
https://www1.nseindia.com/content/indices/ind_niftycommoditieslist.csv
https://www1.nseindia.com/content/indices/ind_niftycpselist.csv
https://www1.nseindia.com/content/indices/ind_niftyenergylist.csv
https://www1.nseindia.com/content/indices/ind_niftyconsumptionlist.csv
https://www1.nseindia.com/content/indices/ind_niftyinfralist.csv
https://www1.nseindia.com/content/indices/ind_niftymnclist.csv
https://www1.nseindia.com/content/indices/ind_niftypselist.csv
https://www1.nseindia.com/content/indices/ind_niftyservicelist.csv
https://www1.nseindia.com/content/indices/ind_Nifty100_Liquid15.csv
https://www1.nseindia.com/content/indices/Nifty_midcap_liquid_15.pdf
df = pd.read_csv('https://archives.nseindia.com/content/historical/EQUITIES/2021/OCT/cm20OCT2021bhav.csv.zip')
https://archives.nseindia.com/products/content/sec_bhavdata_full_20102021.csv
    https://archives.nseindia.com/content/indices/ind_close_all_20102021.csv